# Linear Regression Problem

Building a multiple linear regression model for the prediction of demand for shared bikes using a large dataset on daily bike demands. The project will also aim to determine:
- Which variables are significant in predicting the demand for shared bikes.
- How well those variables describe the bike demands.

To model: 'cnt' as the target variable.


### Constants

Constant values used in the program such as file paths.

In [2]:
PATH = 'Data/day.csv'
PREPROCESSED_FILE = 'Data/pre.csv'
TRAIN_SIZE = 0.8
TEST_SIZE = 0.2
LEARNING_RATE = 0.1
NUM_EPOCHS = 100
L2_LAMBDA = 1e-2

In [3]:
import pandas as pd
import numpy as np

### Dataset Analysis

Simple analysis of the dataset to find the domain of the data to be analyzed.

In [5]:
dataFrame = pd.read_csv(PATH)

In [7]:
dataFrame.shape

(730, 16)

In [19]:
dataFrame.head()

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,01-01-2018,1,0,1,0,6,0,2,14.110847,18.18125,80.5833,10.749882,331,654,985
1,2,02-01-2018,1,0,1,0,0,0,2,14.902598,17.68695,69.6087,16.652113,131,670,801
2,3,03-01-2018,1,0,1,0,1,1,1,8.050924,9.47025,43.7273,16.636703,120,1229,1349
3,4,04-01-2018,1,0,1,0,2,1,1,8.200000,10.60610,59.0435,10.739832,108,1454,1562
4,5,05-01-2018,1,0,1,0,3,1,1,9.305237,11.46350,43.6957,12.522300,82,1518,1600


This is to find the values used in 'weathersit' field. We can see that the value 4 is not available in the data as described in the description file for the dataset.

In [18]:
np.sort(dataFrame['weathersit'].unique())

array([1, 2, 3], dtype=int64)

In [42]:
np.sort(dataFrame['mnth'].unique())

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12], dtype=int64)

In [10]:
dataFrame.isna().sum()

instant       0
dteday        0
season        0
yr            0
mnth          0
holiday       0
weekday       0
workingday    0
weathersit    0
temp          0
atemp         0
hum           0
windspeed     0
casual        0
registered    0
cnt           0
dtype: int64

In [11]:
dataFrame.isnull().sum()

instant       0
dteday        0
season        0
yr            0
mnth          0
holiday       0
weekday       0
workingday    0
weathersit    0
temp          0
atemp         0
hum           0
windspeed     0
casual        0
registered    0
cnt           0
dtype: int64

### Data Preparation

The problem statement document has listed two points under data preparation. The dataset has been verified to have no null or NA values. 

Now, we can convert the 'weathersit' field into 4 unique fields via one-hot encoding technique. Here, the 4 weather status will each get one field which will contain a 1 when present for that status and 0 when not.


In [20]:
pd.get_dummies(dataFrame['weathersit'])

,1,2,3
0,0,1,0
1,0,1,0
2,1,0,0
3,1,0,0
4,1,0,0
...,...,...,...
725,0,1,0
726,0,1,0
727,0,1,0
728,1,0,0


In [22]:
weathersitData = pd.get_dummies(dataFrame['weathersit'])
weathersitData.columns = ['weather_1', 'weather_2', 'weather_3']

In [26]:
weathersitData.head()

,weather_1,weather_2,weather_3
0,0,1,0
1,0,1,0
2,1,0,0
3,1,0,0
4,1,0,0


In [23]:
dataFrame.drop(['weathersit'], axis=1, inplace=True)

In [37]:
processedData = pd.concat([dataFrame.reset_index(drop=True), weathersitData.reset_index(drop=True)], axis=1)

In [92]:
processedData.head()

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,temp,atemp,hum,windspeed,casual,registered,cnt,weather_1,weather_2,weather_3
0,1,01-01-2018,0,0,1,0,6,0,14.110847,18.18125,80.5833,10.749882,331,654,985,0,1,0
1,2,02-01-2018,0,0,1,0,0,0,14.902598,17.68695,69.6087,16.652113,131,670,801,0,1,0
2,3,03-01-2018,0,0,1,0,1,1,8.050924,9.47025,43.7273,16.636703,120,1229,1349,1,0,0
3,4,04-01-2018,0,0,1,0,2,1,8.200000,10.60610,59.0435,10.739832,108,1454,1562,1,0,0
4,5,05-01-2018,0,0,1,0,3,1,9.305237,11.46350,43.6957,12.522300,82,1518,1600,1,0,0


The next fields to have a similar property are the 'mnth' and the 'season' fields. For these we can use binary encoding instead as one-hot would yield too many columns.

We can subtract 1 from 'season' as the field only has values [1,2,3,4] subtracting it would then remove one column when performing binary encoding.

In [55]:
processedData['season'] = processedData['season'] - 1

In [93]:
np.sort(processedData['season'].unique())

array([0, 1, 2, 3], dtype=int64)

In [94]:
import category_encoders as ce

In [95]:
encoder = ce.BinaryEncoder(cols=['mnth', 'season'])

In [96]:
processedDataBin = encoder.fit_transform(processedData)

In [97]:
processedDataBin.head()

,instant,dteday,season_0,season_1,season_2,yr,mnth_0,mnth_1,mnth_2,mnth_3,...,temp,atemp,hum,windspeed,casual,registered,cnt,weather_1,weather_2,weather_3
0,1,01-01-2018,0,0,1,0,0,0,0,1,...,14.110847,18.18125,80.5833,10.749882,331,654,985,0,1,0
1,2,02-01-2018,0,0,1,0,0,0,0,1,...,14.902598,17.68695,69.6087,16.652113,131,670,801,0,1,0
2,3,03-01-2018,0,0,1,0,0,0,0,1,...,8.050924,9.47025,43.7273,16.636703,120,1229,1349,1,0,0
3,4,04-01-2018,0,0,1,0,0,0,0,1,...,8.200000,10.60610,59.0435,10.739832,108,1454,1562,1,0,0
4,5,05-01-2018,0,0,1,0,0,0,0,1,...,9.305237,11.46350,43.6957,12.522300,82,1518,1600,1,0,0


The 'instant' and the 'dteday' columns are not required as a row index is sufficient. 

In [98]:
processedDataBin.drop(['instant', 'dteday'], axis=1, inplace=True)

In [99]:
processedDataBin.head()

,season_0,season_1,season_2,yr,mnth_0,mnth_1,mnth_2,mnth_3,holiday,weekday,...,temp,atemp,hum,windspeed,casual,registered,cnt,weather_1,weather_2,weather_3
0,0,0,1,0,0,0,0,1,0,6,...,14.110847,18.18125,80.5833,10.749882,331,654,985,0,1,0
1,0,0,1,0,0,0,0,1,0,0,...,14.902598,17.68695,69.6087,16.652113,131,670,801,0,1,0
2,0,0,1,0,0,0,0,1,0,1,...,8.050924,9.47025,43.7273,16.636703,120,1229,1349,1,0,0
3,0,0,1,0,0,0,0,1,0,2,...,8.200000,10.60610,59.0435,10.739832,108,1454,1562,1,0,0
4,0,0,1,0,0,0,0,1,0,3,...,9.305237,11.46350,43.6957,12.522300,82,1518,1600,1,0,0


In [100]:
np.sort(processedDataBin['season_0'].unique())

array([0, 1], dtype=int64)

In [110]:
(processedDataBin[(processedDataBin.season_1 == 0) & (processedDataBin.season_2==0) & (processedDataBin.season_0==1)].shape[0] + 
processedDataBin[(processedDataBin.season_1 == 0) & (processedDataBin.season_2==1) & (processedDataBin.season_0==0)].shape[0] + 
processedDataBin[(processedDataBin.season_1 == 1) & (processedDataBin.season_2==0) & (processedDataBin.season_0==0)].shape[0] + 
processedDataBin[(processedDataBin.season_1 == 1) & (processedDataBin.season_2==1) & (processedDataBin.season_0==0)].shape[0]) == processedDataBin.shape[0]

True

This is a surprising result where we expected the binary encoding to yield only two columns which can describe the 4 values but a third column has been generated only for the 0 value. We can verify this by adding the number of records for each type of binary encoding. Hence, we can remove the 'season_0' field as the only time the field has a value of 1 is when the others are 0.

We can also drop the 'casual' and the 'registered' fields as the data is irrelevant and the sum of the two is stored in 'cnt'. This is the target field which we must model.

In [112]:
processedDataBin.drop(['casual', 'registered', 'season_0'], axis=1, inplace=True)

In [113]:
processedDataBin.head()

,season_1,season_2,yr,mnth_0,mnth_1,mnth_2,mnth_3,holiday,weekday,workingday,temp,atemp,hum,windspeed,cnt,weather_1,weather_2,weather_3
0,0,1,0,0,0,0,1,0,6,0,14.110847,18.18125,80.5833,10.749882,985,0,1,0
1,0,1,0,0,0,0,1,0,0,0,14.902598,17.68695,69.6087,16.652113,801,0,1,0
2,0,1,0,0,0,0,1,0,1,1,8.050924,9.47025,43.7273,16.636703,1349,1,0,0
3,0,1,0,0,0,0,1,0,2,1,8.200000,10.60610,59.0435,10.739832,1562,1,0,0
4,0,1,0,0,0,0,1,0,3,1,9.305237,11.46350,43.6957,12.522300,1600,1,0,0


Hence, the data preprocessing is over and now we can proceed to performing analysis.

In [117]:
processedDataBin.to_csv(PREPROCESSED_FILE)

### Split Data

The data is now loaded and split into training and testing sets. We will divide the dateset in a ratio of 8:2

In [2]:
data = pd.read_csv(PREPROCESSED_FILE, header=0, index_col=0)

NameError: name 'pd' is not defined

In [5]:
X = data.drop('cnt', axis=1)
Y = data['cnt'].to_numpy().reshape(-1,1)

In [28]:
X

,season_1,season_2,yr,mnth_0,mnth_1,mnth_2,mnth_3,holiday,weekday,workingday,temp,atemp,hum,windspeed,weather_1,weather_2,weather_3
0,0,1,0,0,0,0,1,0,6,0,14.110847,18.18125,80.5833,10.749882,0,1,0
1,0,1,0,0,0,0,1,0,0,0,14.902598,17.68695,69.6087,16.652113,0,1,0
2,0,1,0,0,0,0,1,0,1,1,8.050924,9.47025,43.7273,16.636703,1,0,0
3,0,1,0,0,0,0,1,0,2,1,8.200000,10.60610,59.0435,10.739832,1,0,0
4,0,1,0,0,0,0,1,0,3,1,9.305237,11.46350,43.6957,12.522300,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
725,0,1,1,1,1,0,0,0,4,1,10.420847,11.33210,65.2917,23.458911,0,1,0
726,0,1,1,1,1,0,0,0,5,1,10.386653,12.75230,59.0000,10.416557,0,1,0
727,0,1,1,1,1,0,0,0,6,0,10.386653,12.12000,75.2917,8.333661,0,1,0
728,0,1,1,1,1,0,0,0,0,0,10.489153,11.58500,48.3333,23.500518,1,0,0


In [6]:
from sklearn.model_selection import train_test_split

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, train_size=TRAIN_SIZE)

### Standardize Data

This is required for using the pytorch models as it can help with performance and accuracy.

In [8]:
from sklearn.preprocessing import StandardScaler

In [9]:
# Standardize the data (mean=0, std=1) using training data
X_scaler = StandardScaler().fit(X_train)
y_scaler = StandardScaler().fit(y_train)

In [10]:
X_train = X_scaler.transform(X_train)
y_train = y_scaler.transform(y_train)
X_test = X_scaler.transform(X_test)
y_test = y_scaler.transform(y_test)

In [11]:
print (f"For TRAIN : mean: {np.mean(X_train, axis=0)[0]:.1f}, std: {np.std(y_train, axis=0)[0]:.1f}")
print (f"For TEST  : mean: {np.mean(X_test, axis=0)[0]:.1f}, std: {np.std(X_test, axis=0)[0]:.1f}")

For TRAIN : mean: 0.0, std: 1.0
For TEST  : mean: 0.0, std: 1.0


### Model

The model is created using pytorch and is configured to be a linear regression type model.

In [12]:
from torch import nn

In [13]:
class LinearRegression(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LinearRegression, self).__init__()
        self.fc1 = nn.Linear(input_dim, output_dim)

    def forward(self, x_in):
        y_pred = self.fc1(x_in)
        return y_pred


In [15]:
# Initialize model
model = LinearRegression(input_dim=X_train.shape[1], output_dim=1)
print (model.named_parameters)

<bound method Module.named_parameters of LinearRegression(
  (fc1): Linear(in_features=17, out_features=1, bias=True)
)>


### Optimizer and Loss Function

In [17]:
from torch.optim import Adam
import torch

In [18]:
optimizer = Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=L2_LAMBDA)
loss_fn = nn.MSELoss()

### Training

In [19]:
# Convert data to tensors
X_train = torch.Tensor(X_train)
y_train = torch.Tensor(y_train)
X_test = torch.Tensor(X_test)
y_test = torch.Tensor(y_test)

In [20]:
# Training
for epoch in range(NUM_EPOCHS):
    # Forward pass
    y_pred = model(X_train)

    # Loss
    loss = loss_fn(y_pred, y_train)

    # Zero all gradients
    optimizer.zero_grad()

    # Backward pass
    loss.backward()

    # Update weights
    optimizer.step()

    if epoch%20==0:
        print (f"Epoch: {epoch} | loss: {loss:.2f}")

Epoch: 0 | loss: 1.45
Epoch: 20 | loss: 0.22
Epoch: 40 | loss: 0.18
Epoch: 60 | loss: 0.17
Epoch: 80 | loss: 0.17


### Evaluation

Evaluate the model using loss function.

In [21]:
# Predictions
pred_train = model(X_train)
pred_test = model(X_test)

# Performance
train_error = loss_fn(pred_train, y_train)
test_error = loss_fn(pred_test, y_test)
print(f"train_error: {train_error:.2f}")
print(f"test_error: {test_error:.2f}")

train_error: 0.17
test_error: 0.16


### Interpretations

Interpretations of the data.


In [23]:
W = model.fc1.weight.data.numpy().squeeze()
b = model.fc1.bias.data.numpy()
print(W)
print(b)

[ 0.11077729 -0.23212485  0.5213363   0.13372508 -0.07280712  0.01253625
  0.02558942 -0.03329861  0.07002633  0.02519902  0.28983164  0.2523073
 -0.11460507 -0.11712451  0.02739543 -0.06696512 -0.15586393]
[-0.00018939]


In [24]:
W_unscaled = W * (y_scaler.scale_/X_scaler.scale_)
b_unscaled = b * y_scaler.scale_ + y_scaler.mean_ - np.sum(W_unscaled*X_scaler.mean_)

print(W_unscaled)
print(b_unscaled)

[  428.18204302  -898.20645786  2014.98944774   521.77250748
  -286.92265801    48.45319398    98.91360861  -372.32247902
    68.35800105   105.79090156    73.92952744    59.31494607
   -15.43041974   -43.03441257   109.87729946  -274.05793959
 -1742.76484456]
[1968.35449542]


season_1	season_2	yr	mnth_0	
mnth_1	mnth_2	mnth_3	holiday	
weekday workingday	temp	atemp	
hum windspeed	weather_1	weather_2	
weather_3

In [29]:
from sklearn.metrics import r2_score

In [33]:
r2_score(y_test, pred_test.detach().numpy())

0.8399213457414538